In [1]:
import torch
import torch.nn as nn

In [2]:
class MLP(nn.Module):
    def __init__(self):
        #调用MLP父类的构造函数来进行必要的初始化。这样在构造实例的时候还可以指定其他参数
        super(MLP,self).__init__()
        self.hidden = nn.Linear(784,256)#隐藏层
        self.act = nn.ReLU()
        self.output = nn.Linear(256,10)#输出层

    def forward(self,x): #定义前向传播函数
        a = self.act(self.hidden(x))
        return self.output(a)

In [3]:
X = torch.rand(2,784)
X

tensor([[0.6551, 0.4523, 0.8342,  ..., 0.6120, 0.0496, 0.4517],
        [0.4287, 0.2133, 0.2635,  ..., 0.5306, 0.4537, 0.8254]])

In [4]:
net = MLP()
net(X)

tensor([[ 0.0690,  0.0268, -0.2331, -0.2221, -0.1315, -0.0137, -0.1474, -0.0389,
         -0.1131,  0.1146],
        [-0.0580,  0.1717, -0.2604, -0.1387, -0.1627, -0.0096, -0.2907, -0.0790,
         -0.1927,  0.2448]], grad_fn=<AddmmBackward0>)

In [5]:
class FancyMLP(nn.Module):
    def __init__(self):
        super(FancyMLP,self).__init__()
        self.rand_weight = torch.randn((20,20),requires_grad=False)#不可训练参数，常数参数
        self.linear = nn.Linear(20,20)

    def forward(self,x):
        x = self.linear(x)
        #使用创建的常数参数以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x,self.rand_weight.data)+1)
        #复用全连接层，等价于两个全连接层共享参数
        x = self.linear(x)
        #控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /=2
        if x.norm().item()<0.8:
            x*=10
        return x.sum()

In [6]:
X = torch.rand(2,20)
net = FancyMLP()
print(net)
net(X)

FancyMLP(
  (linear): Linear(in_features=20, out_features=20, bias=True)
)


tensor(0.5294, grad_fn=<SumBackward0>)

In [7]:
class NestMLP(nn.Module):
    def __init__(self):
        super(NestMLP, self).__init__()
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU())

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

X = torch.rand(2, 40)
print(net)
net(X)

Sequential(
  (0): NestMLP(
    (net): Sequential(
      (0): Linear(in_features=40, out_features=30, bias=True)
      (1): ReLU()
    )
  )
  (1): Linear(in_features=30, out_features=20, bias=True)
  (2): FancyMLP(
    (linear): Linear(in_features=20, out_features=20, bias=True)
  )
)


tensor(3.2189, grad_fn=<SumBackward0>)